## Target

- for each stock:
   - for each date (T):
    - calculate the max price change in next 20 trading dates: price_change = (max{close price in T+1 to T+20} - {close price on T})/({close price on T})


In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [2]:
src_file = 'data/min_date_20120301_163stocks.csv'

In [3]:
df = pd.read_csv(src_file, sep='|', compression='bz2')

In [4]:
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,stock_id
0,2012-03-01,28.594284,28.947300,28.437387,28.868851,3324863,0.0,0.0,A,101
1,2012-03-02,28.875392,29.025751,28.398167,28.581211,2975783,0.0,0.0,A,101
2,2012-03-05,28.502762,28.574673,27.816343,27.966702,4990860,0.0,0.0,A,101
3,2012-03-06,27.646374,27.803269,27.384880,27.639835,5251587,0.0,0.0,A,101
4,2012-03-07,27.764049,28.548530,27.672528,28.411247,4884612,0.0,0.0,A,101


In [5]:
#for each stock_id, get the max close in next 20 trading days
price_col = 'Close'
roll_len=20
new_col = 'next_20day_max'
target_list = []

for stock_id in df['stock_id'].unique():
    df_ = df.loc[df['stock_id']==stock_id, ['stock_id', 'Date', price_col]].copy(deep=True)
    df_.sort_values(by=['Date'], ascending=False, inplace=True)

    df_next20dmax=df_[[price_col]].shift(1).rolling(roll_len).max()
    df_next20dmax.columns=[new_col]
    df_ = df_.merge(df_next20dmax, right_index=True, left_index=True, how='inner')

    df_.dropna(how='any', inplace=True)
    df_['target']= 100*(df_[new_col]-df_[price_col])/df_[price_col]  
    
    target_list.append(df_[['stock_id', 'Date', 'target']])

In [6]:
len(target_list)

170

In [7]:
df_target = pd.concat(target_list)
df_target.shape

(364988, 3)

In [8]:
df_target.tail(5)

,stock_id,Date,target
365989,7,2012-03-07,20.638942
365988,7,2012-03-06,22.376540
365987,7,2012-03-05,18.653357
365986,7,2012-03-02,19.788521
365985,7,2012-03-01,16.789397


In [9]:
df.head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,stock_id
0,2012-03-01,28.594284,28.947300,28.437387,28.868851,3324863,0.0,0.0,A,101
1,2012-03-02,28.875392,29.025751,28.398167,28.581211,2975783,0.0,0.0,A,101
2,2012-03-05,28.502762,28.574673,27.816343,27.966702,4990860,0.0,0.0,A,101
3,2012-03-06,27.646374,27.803269,27.384880,27.639835,5251587,0.0,0.0,A,101
4,2012-03-07,27.764049,28.548530,27.672528,28.411247,4884612,0.0,0.0,A,101


In [10]:
df.shape

(368388, 10)

In [11]:
df = df.merge(df_target, on=['stock_id', 'Date'], how='inner')
df.shape

(364988, 11)

In [12]:
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,stock_id,target
0,2012-03-01,28.594284,28.947300,28.437387,28.868851,3324863,0.0,0.0,A,101,3.849668
1,2012-03-02,28.875392,29.025751,28.398167,28.581211,2975783,0.0,0.0,A,101,4.894805
2,2012-03-05,28.502762,28.574673,27.816343,27.966702,4990860,0.0,0.0,A,101,7.199648
3,2012-03-06,27.646374,27.803269,27.384880,27.639835,5251587,0.0,0.0,A,101,8.467381
4,2012-03-07,27.764049,28.548530,27.672528,28.411247,4884612,0.0,0.0,A,101,5.522314


In [13]:
df.isna().sum()

Date            0
Open            0
High            0
Low             0
Close           0
Volume          0
Dividends       0
Stock Splits    0
ticker          0
stock_id        0
target          0
dtype: int64

In [14]:
src_file

'data/min_date_20120301_163stocks.csv'

In [15]:
dest_file = src_file.replace('.csv', '_w_target.csv')
dest_file

'data/min_date_20120301_163stocks_w_target.csv'

In [16]:
df.to_csv(dest_file, sep='|', compression='bz2', index=False)